In [117]:
import pandas as pd
import re
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

plotly.tools.set_credentials_file(username='mkf', api_key='crwJntc1jJsAdaHKiDPL')

In [3]:
def processing_trades(trades_file):
    pass

In [4]:
def make_chart_df(chart_file):
    pass

In [8]:
def create_chart(chart_df):
    chart_df = pd.read_csv(chart_file, sep=';', decimal=',', header=0)
    chart_df[['Open', 'High', 'Low', 'Close', 'PC']] = chart_df[['Open', 'High', 'Low', 'Close', 'PC']].apply(lambda x:  round(x, 2))
    chart_df.insert(0, 'Date_Time', pd.to_datetime((chart_df['Date'] + ' ' + chart_df['Time']), dayfirst=True))

In [12]:
def draw_trades(trades_df):
    pass

In [14]:
def create_picture(chart_df, trades_df):
    create_chart(chart_df)
    draw_trades(trades_df)

In [24]:
trades_file = ''
chart_file = r'C:\Users\User\Documents\Old_comp\Chart 2016-07-16-09-54.csv'


In [15]:
if __name__ == '__main__':
    trades_df = processing_trades(trades_file)
    chart_df = make_chart_df(chart_file)
    create_picture(chart_df, trades_df)

In [64]:
chart_df = pd.read_csv(chart_file, sep=';', decimal=',', header=0)
chart_df[['Open', 'High', 'Low', 'Close', 'PC']] = chart_df[['Open', 'High', 'Low', 'Close', 'PC']].apply(lambda x:  round(x, 2))
chart_df.insert(0, 'Date_Time', pd.to_datetime((chart_df['Date'] + ' ' + chart_df['Time']), dayfirst=True))

In [66]:
chart_df.head()

,Date_Time,Date,Time,Open,High,Low,Close,PC
0,2016-07-15 15:55:00,15.07.2016,15:55:00,215.80,215.96,215.79,215.81,216.12
1,2016-07-15 15:50:00,15.07.2016,15:50:00,215.84,215.90,215.79,215.80,216.12
2,2016-07-15 15:45:00,15.07.2016,15:45:00,215.89,215.94,215.83,215.83,216.12
3,2016-07-15 15:40:00,15.07.2016,15:40:00,216.04,216.04,215.88,215.90,216.12
4,2016-07-15 15:35:00,15.07.2016,15:35:00,216.06,216.09,215.98,216.03,216.12


In [75]:
trace = go.Candlestick(x=chart_df.Date_Time,
                       open=chart_df.Open,
                       high=chart_df.High,
                       low=chart_df.Low,
                       close=chart_df.Close)
data = [trace]
#py.iplot(data)

In [269]:
excel_df = pd.read_excel(r'C:/Users/User/Downloads/07060212-2018-08-09-to-2018-08-09-detailed.xls', encoding='ASCII', header=None)

In [270]:
file_parsing = {}
reindex = {a : b for a, b in zip(range(22), excel_df.iloc[2, :])}
excel_df = excel_df.rename(columns=reindex)
#stock_execution = pd.DataFrame()
for i in range(len(excel_df)):
    if pd.notna(excel_df.iloc[i][0]):
        if re.match(r'\d{1,2}/\d{1,2}/\d{4}', excel_df.iloc[i][0]):
            date = excel_df.iloc[i][0]
            file_parsing[date] = {}
            continue
        
        #ticker_match = re.match(r'^(\w+)\s{1,2}-\s.*', excel_df.iloc[i][0])
        elif re.match(r'^(\w+)\s{1,2}-\s.*', excel_df.iloc[i][0]):
            ticker = re.match(r'^(\w+)\s{1,2}-\s.*', excel_df.iloc[i][0])[1]
            file_parsing[date][ticker] = pd.DataFrame()
            continue
        
        elif excel_df.iloc[i][0] == 'Time':
            
            file_parsing[date][ticker].reindex(columns=excel_df.iloc[i])
            columns = excel_df.iloc[i]
            continue
        
        #time_match = re.match(r'\d\d:\d\d:\d\d', excel_df.iloc[i][0])
        elif re.match(r'\d\d:\d\d:\d\d', excel_df.iloc[i][0]):
            file_parsing[date][ticker] = file_parsing[date][ticker].append(excel_df.iloc[i, :])

In [238]:
excel_df.iloc[2, :]

0         Time
1     Order Id
2      Fill Id
3        Route
4          Liq
5          B/S
6          Qty
7        Price
8     Position
9        Gross
10        Comm
11     Ecn Fee
12         SEC
13         ORF
14         TAF
15         NFA
16        NSCC
17         Acc
18         Clr
19        Misc
20         Net
21         NaN
Name: 2, dtype: object

In [137]:
text = 'EAF  - GRAFTECH INTERNATIONAL LTD'
re.match(r'^(\w+)\s{1,2}-\s.*', text)[1]

'EAF'

In [203]:
excel_df = excel_df.reindex(columns=excel_df.iloc[2, :])

In [272]:
file_parsing['8/9/2018']['BKNG']

,nan,Acc,B/S,Clr,Comm,Ecn Fee,Fill Id,Gross,Liq,Misc,NFA,NSCC,Net,ORF,Order Id,Position,Price,Qty,Route,SEC,TAF,Time
25,NaN,0.0,T,0.0,0.135,0.09,NaN,0.0,RI,0.0,0.0,0.01,-0.985,0.0,101993,-30.0,1889.78,30.0,ARCA,0.74,0.01,08:58:26
26,NaN,0.0,T,0.0,0.045,-0.02,NaN,0.0,A,0.0,0.0,0.01,-0.295,0.0,102593,-40.0,1890.76,10.0,ARCA,0.25,0.01,09:00:49
27,NaN,0.0,B,0.0,0.045,-0.02,NaN,-153.2,A,0.0,0.0,0.01,-153.235,0.0,107223,-30.0,1905.10,10.0,ARCA,0.00,0.00,09:23:03
28,NaN,0.0,B,0.0,0.045,-0.02,NaN,-108.7,A,0.0,0.0,0.01,-108.735,0.0,106266,-20.0,1900.65,10.0,ARCA,0.00,0.00,09:24:52
29,NaN,0.0,B,0.0,0.045,-0.02,NaN,-108.7,A,0.0,0.0,0.01,-108.735,0.0,106267,-10.0,1900.65,10.0,ARCA,0.00,0.00,09:24:52
30,NaN,0.0,B,0.0,0.045,0.03,NaN,-318.0,RT,0.0,0.0,0.01,-318.085,0.0,120744,0.0,1922.56,10.0,EDGX,0.00,0.00,09:32:46
31,NaN,0.0,B,0.0,0.045,-0.02,NaN,0.0,A,0.0,0.0,0.01,-0.035,0.0,123365,10.0,1935.10,10.0,ARCA,0.00,0.00,09:37:09
32,NaN,0.0,T,0.0,0.045,-0.02,NaN,-179.3,A,0.0,0.0,0.01,-179.595,0.0,124970,0.0,1917.17,10.0,ARCA,0.25,0.01,09:40:47


In [192]:
df = pd.DataFrame()
df = df.reindex(columns=['a', 'b'])
df

,a,b


In [267]:
list(range(21,-1,-1))

[21, 20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0]

In [265]:
excel_df.iloc[2, :]

0         Time
1     Order Id
2      Fill Id
3        Route
4          Liq
5          B/S
6          Qty
7        Price
8     Position
9        Gross
10        Comm
11     Ecn Fee
12         SEC
13         ORF
14         TAF
15         NFA
16        NSCC
17         Acc
18         Clr
19        Misc
20         Net
21         NaN
Name: 2, dtype: object